# 前言

主要参考：

https://github.com/richliao/textClassifier

参考其博客：

https://richliao.github.io/supervised/classification/2016/12/26/textclassifier-HATN/

主要使用数据是IMDB：

https://www.kaggle.com/c/word2vec-nlp-tutorial/data  （到这里下载）

----
**运行前提：**
参考 E:\jupter\Blog_Study\深度框架学习\theano安装 这个文档，Anaconda默认环境是3.6 这不影响，但是3.6默认安装的keras是2.2.x,而这个代码是2.0.8 想要正确运行，就需要先卸载2.2版本的keras 去使用2.0.8的，python版本倒不是很重要。台式机是配置好的， 在HAAT_Attention文件夹下，py3.x的版本正确，以后可以用那个运行

# 先根据博客走一遍

> 在实现层次注意力机制网络（那个论文的模型）之前，先实现一个层次LSTM网络作为基线。想要实现层次LSTM，需要将数据构建成3D而不是以前的2D

## 处理数据

In [1]:
import numpy as np
import pandas as pd
import sys,os

In [2]:
data_train=pd.read_csv('../labeledTrainData.tsv',sep='\t')

In [3]:
data_train.head(2)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."


>+ 这里要使用句子级别，构建了一个三维数组，文档个数X最大句子数X每句最大长度  本例中= 25000X15X100
+ 如果是以前词级别的话，输入的数据应该是 文档个数X文档最大长度，25000X250 TensorFlow教程里是这个维度
+ data[i,j,k] i表示review序号，[0,len(reviews)],j表示句子序号，[1,MAX_SENTS],k表示词语序号[0,MAX_NB_WORDS]

In [4]:
from nltk import tokenize

In [5]:
import re
def clean_str(string):
    string=re.sub(r"\\","",string)
    string=re.sub(r"\'","",string)
    string=re.sub(r'\"','',string)
    return string.strip().lower()

In [6]:
data_train.review.shape[0],data_train.shape

(25000, (25000, 3))

### 分句

In [7]:
stringa='''The  (Salamandra salamandra)? an amphibian found. Its black the head and back, toxins.'''
tokenize.sent_tokenize(stringa)

['The  (Salamandra salamandra)?',
 'an amphibian found.',
 'Its black the head and back, toxins.']

看上个cell可以知道，sent_tokenize的标准是 句号,问号，其他标点符号都不分的，  

+ 关于英文分句 https://zhuanlan.zhihu.com/p/41804488  手动划分 句号 叹号 问号 貌似暂时知道的就这三种符号比较常见
+ 关于中文分句，
    + 看一下结巴分词行不行，（查看后知道，不行）
    + 更多是使用正则表达式或者string的split，自己固定要分的标点符号，我觉得这样比较好吧。毕竟微博语言并不规整，很多人不怎么好好用标点符号的。
    + 参考：
        + （采用）https://blog.csdn.net/blmoistawinde/article/details/82379256#commentBox  
        + （借鉴理解）https://blog.csdn.net/zhuzuwei/article/details/80487032

In [8]:
zh_string='巨无霸锯床，懂不？加工金属的，3米厚的金属板一下下去像裁布似的，莫名喜欢[色]，全淘男友力max排行榜上有名'
sentences = re.split('(。|！|\!|\.|？|\?|，)',zh_string) 
'''
只进行split操作会保留那些标点符号，所以需要额外操作来只提取文本
'''
new_sents = []
for i in range(int(len(sentences)/2)):
    sent = sentences[2*i] + sentences[2*i+1]
    '''
    这里的假设是 分句 标点符号 分句 标点符号 有点理想化，但是还可以吧
    '''
    new_sents.append(sent)

In [9]:
new_sents

['巨无霸锯床，', '懂不？', '加工金属的，', '3米厚的金属板一下下去像裁布似的，', '莫名喜欢[色]，']

In [10]:
import re
def cut_sent(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para)  # 英文省略号
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  # 中文省略号
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    # 如果双引号前有终止符，那么双引号才是句子的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
    para = para.rstrip()  # 段尾如果有多余的\n就去掉它
    # 很多规则中会考虑分号;，但是这里我把它忽略不计，破折号、英文双引号等同样忽略，需要的再做些简单调整即可。
    return para.split("\n")

In [11]:
cut_sent(zh_string)

['巨无霸锯床，懂不？', '加工金属的，3米厚的金属板一下下去像裁布似的，莫名喜欢[色]，全淘男友力max排行榜上有名']

### 得到句子集 评论集 标签集

+ reviews的数据结构是 [['I love you','I love you too;],['I love dog','I love sunshine'],.....]，列表嵌入列表，reviews的每个元素是一个列表sentenceslist，sentenceslist中是每条评论分句后的列表
+ texts是经过处理的问题，是[],其中列表的每个元素就是一整条评论
+ labels 列表，二分类

In [12]:
from bs4 import  BeautifulSoup
reviews=[]
texts=[]
labels=data_train.sentiment.values
for idx in range(data_train.review.shape[0]):
    text=BeautifulSoup(data_train.review[idx],'lxml')
    text=clean_str(text.get_text())
    texts.append(text)
    sentences=tokenize.sent_tokenize(text)
    '''
    把text分成句子,参照上个cell
    '''
    reviews.append(sentences)

In [13]:
# reviews[2],labels[2],texts[2]
# 这个数据集文本太长了，稍微看一下就好

### 定义几个全局变量

In [14]:
MAX_SENT_LENGTH=100
MAX_SENTS=10
MAX_NB_WORDS=20000  #这边参数改一改，不然我这电脑跑不动
EMBEDDING_DIM=300
VALIDATION_SPLIT=0.2

### 进行分词，词表构建和向量化

In [15]:
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


>Text tokenization utility class.<br>
This class allows to vectorize a text corpus, by turning each text into either a sequence of integers (each integer being the index of a token in a dictionary) or into a vector where the coefficient for each token could be binary, based on word count, based on tf-idf...<br>
Tokenizer类可以用来向量化一个文本语料，通过将每个文本变成整数序列，或者变成一个每个元素是二进制表示的向量（one-hot编码）。

In [16]:
from keras import backend as K
import os
import importlib  #python3 reload模块 reload不再是内建函数 需要显式调用

def set_keras_backend(backend):
    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        importlib.reload(K)
        assert K.backend() == backend

set_keras_backend("theano")  #如果后端不是theano 就换成theano
import theano

Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [17]:
tokenizer=Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts) # Updates internal vocabulary based on a list of texts. 要求参数是一个list
# tokenizer会构建一个词典 id映射字典

In [18]:
tokenizer.word_index['happy']

642

In [19]:
text_to_word_sequence('The  (Salamandra salamandra)? an amphibian found. Its black the head and back, toxins.')

['the',
 'salamandra',
 'salamandra',
 'an',
 'amphibian',
 'found',
 'its',
 'black',
 'the',
 'head',
 'and',
 'back',
 'toxins']

In [20]:
'''
Converts a text to a sequence of words (or tokens).分词 但是对象必须是一个string，不能是列表了，所以只能一个句子一个句子分
'''
data=np.zeros((len(texts),MAX_SENTS,MAX_SENT_LENGTH),dtype='int32')
for i,sentences in enumerate(reviews):  #对每条评论
    for j,sent in enumerate(sentences):  #对每条评论中的每个分句
        if j < MAX_SENTS:  #直接取前10个句子，不考虑什么情感词
            wordTokens=text_to_word_sequence(sent) 
            k=0 #用来计数分句的分词数
            for _,word in enumerate(wordTokens):  #对这个分句的分词
                if k<MAX_SENT_LENGTH and tokenizer.word_index[word]<MAX_NB_WORDS:
                    # 保证每个分句的词个数 即分句长度不超过100
                    #保证这个分句里的词 位于 词频的前2w位
                    data[i,j,k]=tokenizer.word_index[word]
                    k=k+1       

In [21]:
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Total 81501 unique tokens.
Shape of data tensor: (25000, 10, 100)
Shape of label tensor: (25000, 2)
Number of positive and negative reviews in traing and validation set
[10020.  9980.]
[2480. 2520.]


In [22]:
GLOVE_DIR='../../TheSecond-Paper/word_embedding/en_model.txt'
embedding_index={}
f=open(GLOVE_DIR)
for  line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float64')
    embedding_index[word]=coefs
f.close()

print('Total %s word vectors.' % len(embedding_index))

embedding_matrix=np.random.random((len(tokenizer.word_index)+1,EMBEDDING_DIM)) 
for word,i in tokenizer.word_index.items():   #词 词id
    embedding_vector=embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector  #词id是从0开始的 词向量

Total 209513 word vectors.


In [23]:
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

In [24]:
class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [25]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=True,
                            mask_zero=True)
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer(100)(l_lstm)
sentEncoder = Model(sentence_input, l_att)

review_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(review_encoder)
l_att_sent = AttLayer(100)(l_lstm_sent)
preds = Dense(2, activation='softmax')(l_att_sent)
model = Model(review_input, preds)

In [26]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [27]:
print("model fitting - Hierachical attention network")
model.fit(x_train[:400], y_train[:400], validation_data=(x_val, y_val),epochs=10, batch_size=50)

model fitting - Hierachical attention network


D:\anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 400 samples, validate on 5000 samples
Epoch 1/10
400/400 [==============================] - 197s - loss: 0.7518 - acc: 0.5375 - val_loss: 0.6893 - val_acc: 0.6088
Epoch 2/10
400/400 [==============================] - 196s - loss: 0.6766 - acc: 0.5825 - val_loss: 0.6951 - val_acc: 0.4960
Epoch 3/10
400/400 [==============================] - 191s - loss: 0.6673 - acc: 0.5950 - val_loss: 0.6680 - val_acc: 0.6278
Epoch 4/10
400/400 [==============================] - 189s - loss: 0.5821 - acc: 0.7725 - val_loss: 0.6912 - val_acc: 0.5258
Epoch 5/10
400/400 [==============================] - 195s - loss: 0.3914 - acc: 0.8575 - val_loss: 0.9110 - val_acc: 0.5464
Epoch 6/10
400/400 [==============================] - 191s - loss: 0.1370 - acc: 0.9575 - val_loss: 0.7450 - val_acc: 0.7176
Epoch 7/10
400/400 [==============================] - 189s - loss: 0.0214 - acc: 0.9975 - val_loss: 0.6469 - val_acc: 0.7740
Epoch 8/10
400/400 [==============================] - 192s - loss: 0.0051 - ac

+ 配置安装theano： https://zhyack.github.io/posts/2016_05_26-Configurate-Theano-On-Windows.html
+ WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
    + https://blog.csdn.net/m0_38058163/article/details/80657447
+ http://deeplearning.net/software/theano/library/config.html#envvar-THEANO_FLAGS  <br>
    config.blas.ldflags[source]
    
    Default: '-lblas'
    
    Link arguments to link against a (Fortran) level-3 blas implementation. The default will test if '-lblas' works. If not, we will disable our C code for BLAS.